## This notebook explores colors using the different flux measurements

#### Standard imports

In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
from rail.raruma import utility_functions as raruma_util
from rail.raruma import plotting_functions as raruma_plot

#### Change this to get the right location

In [ ]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [ ]:
t = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0/5063/object.parquet')

#### Define data quality masks

In [ ]:
blendedness = t['i_blendedness'].to_numpy()
blendedness = np.where(np.isnan(blendedness), 0., blendedness).clip(0., 1.)
mask  = np.bitwise_and(
    np.bitwise_and(
        t['i_extendedness'].to_numpy() == 1,
        t['g_psfFlux'].to_numpy() / t['g_psfFluxErr'].to_numpy() > 10,
    ),
    np.bitwise_and(
        t['i_psfFlux'].to_numpy() / t['i_psfFluxErr'].to_numpy() > 10,
        t['r_psfFlux'].to_numpy() / t['r_psfFluxErr'].to_numpy() > 10,
        t['z_psfFlux'].to_numpy() / t['z_psfFluxErr'].to_numpy() > 10,
    ),
)


#### Get the magnitudes

In [ ]:
mags_psf = raruma_util.make_mag_dict(t, '{band}_psfFlux', 'griz')
mags_cModel = raruma_util.make_mag_dict(t, '{band}_cModelFlux', 'griz')
mags_sersic = raruma_util.make_mag_dict(t, '{band}_sersicFlux', 'griz')
mags_1p0 = raruma_util.make_mag_dict(t, '{band}_gaap1p0Flux', 'griz')
mags_3p0 = raruma_util.make_mag_dict(t, '{band}_gaap3p0Flux', 'griz')

#### Make some plots

In [ ]:
#raruma_plot.plot_mag_mag_scatter(mags_cModel, mags_psf, 'griz')

In [ ]:
#raruma_plot.plot_mag_mag_scatter(mags_cModel, mags_psf, 'griz')

In [ ]:
#raruma_plot.plot_mag_mag_scatter(mags_sersic, mags_psf, 'griz')

In [ ]:
#raruma_plot.plot_mag_mag_scatter(mags_sersic, mags_psf, 'griz')

In [ ]:
raruma_plot.plot_mag_mag_scatter(mags_1p0, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_resid(mags_1p0, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_resid(mags_3p0, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_scatter(mags_3p0, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_scatter(mags_sersic, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_resid(mags_sersic, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_scatter(mags_cModel, mags_psf, 'griz', mask=mask)

In [ ]:
raruma_plot.plot_mag_mag_resid(mags_cModel, mags_psf, 'griz', mask=mask)

In [ ]:
colors_psf = raruma_util.make_color_dict(t, '{band}_psfFlux', 'griz')
colors_cModel = raruma_util.make_color_dict(t, '{band}_cModelFlux', 'griz')
colors_1p0 = raruma_util.make_color_dict(t, '{band}_gaap1p0Flux', 'griz')
colors_3p0 = raruma_util.make_color_dict(t, '{band}_gaap3p0Flux', 'griz')
colors_sersic = raruma_util.make_color_dict(t, '{band}_sersicFlux', 'griz')

In [ ]:
raruma_plot.plot_color_resid(colors_cModel, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
raruma_plot.plot_color_scatter(colors_cModel, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
raruma_plot.plot_color_scatter(colors_3p0, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask, norm="log")

In [ ]:
raruma_plot.plot_color_resid(colors_cModel, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
raruma_plot.plot_color_resid(colors_1p0, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
raruma_plot.plot_color_resid(colors_sersic, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask)

In [ ]:
raruma_plot.plot_color_resid(colors_3p0, colors_psf, ['g-r', 'r-i', 'i-z'], mask=mask, norm="log")